In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Im Accessing this dataset with my gdrive because I had to clean the data by hand and reupload to model.

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
https://drive.google.com/drive/folders/1RpHYaTL-r22AUqhDhQRjQAxm0MzxGERf?usp=drive_link

In [2]:
folder_id = '1RpHYaTL-r22AUqhDhQRjQAxm0MzxGERf'  

file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

for file in file_list:
    print(f"File Name: {file['title']}, File ID: {file['id']}")

File Name: Training_images.zip, File ID: 1LPRXi81BdrLT4u7UE5etSqg-JfwjUkNk
File Name: Test_images.zip, File ID: 1Q7IgKDLd1270N9x1mxCrcVPm81IoX6kK
File Name: Validation_images.zip, File ID: 1__8UfmITWqJ8PJbVCK33pbwsYvz9YZKh


In [3]:
import os

os.makedirs('downloaded_files', exist_ok=True)

for file in file_list:
    if file['title'].endswith('.zip'): 
        print(f"Downloading {file['title']}...")
        downloaded = drive.CreateFile({'id': file['id']})
        downloaded.GetContentFile(os.path.join('downloaded_files', file['title']))
        print(f"Downloaded {file['title']} successfully!")

Downloaded Training_images.zip successfully!
Downloaded Test_images.zip successfully!
Downloaded Validation_images.zip successfully!


You can Ignore the blocks above

In [4]:
import zipfile
import os
import random
import shutil
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import classification_report
import numpy as np

Extracting the zip files

In [5]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

extract_zip('/kaggle/working/downloaded_files/Training_images.zip', '/kaggle/working/train')
extract_zip('/kaggle/working/downloaded_files/Validation_images.zip', '/kaggle/working/val')
extract_zip('/kaggle/working/downloaded_files/Test_images.zip', '/kaggle/working/test')


Creating positive and negative paths for our Classification Problem

In [6]:
positive_path = '/kaggle/working/train/Training_images/Positive'
negative_path = '/kaggle/working/train/Training_images/Negative'
balanced_train_path = '/kaggle/working/balanced_training'

os.makedirs(balanced_train_path, exist_ok=True)
os.makedirs(os.path.join(balanced_train_path, 'positive'), exist_ok=True)
os.makedirs(os.path.join(balanced_train_path, 'negative'), exist_ok=True)

# Select 2304 positive images randomly
# Here im matching the number of pos and neg images for better accuracy
positive_images = [os.path.join(positive_path, f) for f in os.listdir(positive_path) 
                  if f.endswith('.jpg') or f.endswith('.png')]
random.shuffle(positive_images)
selected_positive = positive_images[:2304]

for img in selected_positive:
    shutil.copy(img, os.path.join(balanced_train_path, 'positive'))

negative_images = [os.path.join(negative_path, f) for f in os.listdir(negative_path) 
                   if f.endswith('.jpg') or f.endswith('.png')]
for img in negative_images:
    shutil.copy(img, os.path.join(balanced_train_path, 'negative'))

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


Augmenting to generate more pictures based on our current data 

In [47]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.RandomAdjustSharpness(2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


Data Loaders

In [48]:
train_dataset = ImageFolder('/kaggle/working/train/Training_images', transform=train_transform)
val_dataset = ImageFolder('/kaggle/working/val/Validation_images', transform=val_test_transform)
test_dataset = ImageFolder('/kaggle/working/test/Test_images', transform=val_test_transform)

In [50]:
from sklearn.utils.class_weight import compute_class_weight
# Calculate class weights
class_weights = compute_class_weight(
    'balanced', 
    classes=[0, 1], 
    y=train_dataset.targets
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=4, pin_memory=True)

using resnet50 for our model. I personally chose this model because we don't have a huge amount of data and hence the modeling progress shouldn't take long and we can achieve better accuracy. Else we can just go for less layers like resnet18.

In [51]:
model = models.resnet50(pretrained=True)

# Replacing final layer with dropout
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 2)
)

model = model.to(device)

# Enhanced optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2):
        super().__init__()
        self.alpha = alpha  # Weight for positive class
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

criterion = FocalLoss(alpha=0.85)  # Prioritize positive class
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', 
                                                factor=0.1, patience=3)
scaler = torch.cuda.amp.GradScaler()  # For mixed precision

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-51-9ae16284a56c>:32: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # For mixed precision


########

Train and Val

In [52]:
best_val_acc = 0.0
patience = 5
epochs_without_improve = 0
train_losses = []
val_accuracies = []

for epoch in range(50):  
    print(f'\nEpoch {epoch+1}/50')
    
    # Training phase
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc='Training')
    
    for inputs, labels in progress_bar:
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        # Mixed precision training
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * inputs.size(0)
        progress_bar.set_postfix({'loss': loss.item()})
    
    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)
    
    # Val
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validating'):
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    val_acc = accuracy_score(val_labels, val_preds)
    val_accuracies.append(val_acc)
    scheduler.step(val_acc)
    
    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_without_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'New best model saved! Val Acc: {val_acc:.4f}')
    else:
        epochs_without_improve += 1
        if epochs_without_improve >= patience:
            print(f'\nEarly stopping at epoch {epoch+1}')
            break
    
    print(f'Epoch {epoch+1} Summary:')
    print(f'Train Loss: {epoch_loss:.4f} | Val Acc: {val_acc:.4f}')
    print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')


Epoch 1/50


Training:   0%|          | 0/73 [00:00<?, ?it/s]<ipython-input-52-d28880c93c9f>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


New best model saved! Val Acc: 0.7400
Epoch 1 Summary:
Train Loss: 0.1494 | Val Acc: 0.7400
Current LR: 0.001000

Epoch 2/50


Training:   0%|          | 0/73 [00:00<?, ?it/s]<ipython-input-52-d28880c93c9f>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 2/2 [00:00<00:00,  2.28it/s]


Epoch 2 Summary:
Train Loss: 0.1129 | Val Acc: 0.7100
Current LR: 0.001000

Epoch 3/50


...

Epoch 9 Summary:
Train Loss: 0.1062 | Val Acc: 0.8000
Current LR: 0.001000

Epoch 10/50


Training:   0%|          | 0/73 [00:00<?, ?it/s]<ipython-input-52-d28880c93c9f>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 2/2 [00:00<00:00,  2.33it/s]


Epoch 10 Summary:
Train Loss: 0.0992 | Val Acc: 0.7500
Current LR: 0.000100

Epoch 11/50


Training:   0%|          | 0/73 [00:00<?, ?it/s]<ipython-input-52-d28880c93c9f>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Validating: 100%|██████████| 2/2 [00:00<00:00,  2.19it/s]


Early stopping at epoch 11


Using Early stopping for when the model isn't improving to save time and avoid overfitting our model

Test Eval

In [63]:
# Loading the best model
model.load_state_dict(torch.load('best_model.pth'))

# Test evaluation
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc='Testing'):
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        positive_probs = probs[:, 1]  # Probability of CAD
        
        new_threshold = 0.7  # to reduce false positives
        preds = (positive_probs > new_threshold).long()
        
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

print('\nFinal Test Metrics:')
print(f'Accuracy: {accuracy_score(test_labels, test_preds):.4f}')
print(classification_report(test_labels, test_preds, target_names=['Negative', 'Positive']))
print('Confusion Matrix:')
print(confusion_matrix(test_labels, test_preds))

<ipython-input-63-b151588d2e69>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
Testing: 100%|██████████| 19/19 [00:04<0


Final Test Metrics:
Accuracy: 0.9421
              precision    recall  f1-score   support

    Negative       0.95      0.99      0.97      1066
    Positive       0.83      0.57      0.67       125

    accuracy                           0.94      1191
   macro avg       0.89      0.78      0.82      1191
weighted avg       0.94      0.94      0.94      1191

Confusion Matrix:
[[1051   15]
 [  54   71]]


In [ ]:
'''
Final Test Metrics (incase it's not shown in github):
Accuracy: 0.9421
              precision    recall  f1-score   support

    Negative       0.95      0.99      0.97      1066
    Positive       0.83      0.57      0.67       125

    accuracy                           0.94      1191
   macro avg       0.89      0.78      0.82      1191
weighted avg       0.94      0.94      0.94      1191

Confusion Matrix:
[[1051   15]

'''

as you can see since we don't have enough positive cases for our data we have a 0.57 recall on our positive cases which means 43% of our positive cases are missed. hence this model isn't good for real life deployment unless we add more and equal data to it. but it's meant for practice anyways.